In [2]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark


# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"


# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages
Get:12 https://developer.downl

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-04-21 01:30:25--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.21MB/s    in 0.8s    

2021-04-21 01:30:27 (1.21 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Camera_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Camera_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    2975964|R1NBG94582SJE2|B00I01JQJM|     860486164|GoPro Rechargeabl...|          Camera|          5|            0|          0|   N|                Y|          Five Stars|                  ok| 2015-08-31|
|         US|   23526356|R273DCA6Y0H9V7|B00TCO0ZAA|     292641483|Professional 58mm...|          Camera|          5|    

In [6]:
#filter data and create a new dataframe with total_votes >= 20
vine_df= df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vote_more_than_20= df.filter('total_votes >= 20')
vote_more_than_20.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    3200070|R3FJ319XA6ZAUQ|B00ZI7IH1E|     450586100|Funlux? 8CH Full ...|          Camera|          2|           21|         25|   N|                Y|            Mediocre|The product is gr...| 2015-08-31|
|         US|   40768960| R6HRF25HUMIIE|B00TAG8F52|     482310914|LimoStudio Digita...|          Camera|          5|    

In [7]:
#filter all rows where the number of helpful_votes divided by total_votes is greater than 50%
vine_greater_than_50_df= vote_more_than_20.filter(vote_more_than_20['helpful_votes']/vote_more_than_20['total_votes'] >= 0.5)
vine_greater_than_50_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    3200070|R3FJ319XA6ZAUQ|B00ZI7IH1E|     450586100|Funlux? 8CH Full ...|          Camera|          2|           21|         25|   N|                Y|            Mediocre|The product is gr...| 2015-08-31|
|         US|   40768960| R6HRF25HUMIIE|B00TAG8F52|     482310914|LimoStudio Digita...|          Camera|          5|    

In [8]:
#filter for reviews included in the vine program
vine_included_df= vine_greater_than_50_df.filter(vine_greater_than_50_df['vine']== 'Y')
vine_included_df.show() 

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   38607681|R37L6YZDX0P3J9|B00SEKXJ6U|     408705341|Guardzilla  All-I...|          Camera|          1|           28|         35|   Y|                N|Gave up after an ...|I have installed ...| 2015-08-29|
|         US|   52532594|R2JT4XRQO2KASL|B00SEKXJ6U|     408705341|Guardzilla  All-I...|          Camera|          5|    

In [9]:
#filter for reviews not included in the vine program
vine_not_included_df= vine_greater_than_50_df.filter(vine_greater_than_50_df['vine']== 'N')
vine_not_included_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    3200070|R3FJ319XA6ZAUQ|B00ZI7IH1E|     450586100|Funlux? 8CH Full ...|          Camera|          2|           21|         25|   N|                Y|            Mediocre|The product is gr...| 2015-08-31|
|         US|   40768960| R6HRF25HUMIIE|B00TAG8F52|     482310914|LimoStudio Digita...|          Camera|          5|    

In [10]:
#determine the total number of reviews
vine_greater_than_50_df.count()


51129

In [11]:
#determine the number of 5-star reviews
five_star_review_df= vine_greater_than_50_df.filter(vine_greater_than_50_df['star_rating']== 5)
five_star_review_df.count()

25477

In [12]:
#percentage of 5-star paid reviews
five_star_review_df.filter(five_star_review_df['vine']== 'Y').count()/vine_greater_than_50_df.filter(vine_greater_than_50_df['vine']== 'Y').count()

0.4233937397034596

In [13]:
#percentage of 5-star unpaid reviews
five_star_review_df.filter(five_star_review_df['vine']== 'N').count()/vine_greater_than_50_df.filter(vine_greater_than_50_df['vine']== 'N').count()

0.4991884723486798

In [14]:
five_star_review_df.filter(five_star_review_df['vine']== 'Y').count()

257

In [15]:
five_star_review_df.filter(five_star_review_df['vine']== 'N').count()

25220

In [16]:
vine_greater_than_50_df.filter(vine_greater_than_50_df['vine']== 'Y').count()

607

In [17]:
vine_greater_than_50_df.filter(vine_greater_than_50_df['vine']== 'N').count()

50522